In [1]:
from utils.utils import build_market_image, prepare_forecast_data
# from environments.live_environments import LiveTradingEnv
from utils.clients import CoinbaseClient
from Keys import *
# from utils.mappings import oanda_time_map
from utils.reward_functions import sharpe_reward_function
import pandas as pd
from json import dumps

import time
import datetime



need to install oandapyV20
need to install alpaca


In [2]:
# from neuralforecast.core import NeuralForecast
# forecasting_model=NeuralForecast.load('MultiHeadForecastingModel/')
# agent_dir='Agent/final_checkpoints/policies/default_policy'
# agent= Policy.from_checkpoint(agent_dir)

In [3]:
base_asset='DOGE'
quote_asset='USDC'
test_net=False
time_frame='1h'
product_type='FUTURE'
futures_target='DOG-29NOV24-CDE'
exchange='coinbase'
trade_target='/'.join([base_asset,quote_asset]) if product_type=='SPOT' else  futures_target

positions = [-1,0,1]
trade_target,product_type

('DOG-29NOV24-CDE', 'FUTURE')

In [4]:
trade_target

'DOG-29NOV24-CDE'

In [5]:
client=CoinbaseClient(api_key=coinbase_api_key,
                    api_secret=coinbase_api_secret,
                
                    paper=test_net,
                    symbol=trade_target,
                    time_frame=time_frame,
                    product_type=product_type

                    )
client.trade_client.base_url,client.base_asset,client.quote_asset,client.symbol,client.time_frame



Paper TradingFalse on coinbase at api.coinbase.com
['DOG', '29NOV24', 'CDE']


('api.coinbase.com', 'DOG-29NOV24-CDE', 'USD', 'DOG-29NOV24-CDE', '1h')

In [6]:
product_frame=client.get_products(product_type='FUTURE')
product_frame

,product_id,price,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,base_max_size,base_name,quote_name,...,contract_size,contract_root_unit,group_description,contract_expiry_timezone,group_short_description,contract_expiry_type,contract_display_name,time_to_expiry_ms,contract_expiry_name,twenty_four_by_seven
0,BIT-29NOV24-CDE,90040,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.01,BTC,Nano Bitcoin Futures,Europe/London,Nano BTC,EXPIRING,BTC 29 NOV 24,1198301365,2024-11,False
1,BIT-27DEC24-CDE,90510,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.01,BTC,Nano Bitcoin Futures,Europe/London,Nano BTC,EXPIRING,BTC 27 DEC 24,3617501365,2024-12,False
2,ET-29NOV24-CDE,3046.5,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.1,ETH,Nano Ether Futures,Europe/London,Nano ETH,EXPIRING,ETH 29 NOV 24,1198301365,2024-11,False
3,GOL-27NOV24-CDE,2570.9,1,0.01,0,100000000,1,100000000,,US Dollar,...,1,CDEGLD,Gold Futures,Europe/London,Gold Futures,EXPIRING,GLD 26 NOV 24,948101365,2024-12,False
4,BCH-29NOV24-CDE,425.4,1,0.01,0,100000000,1,100000000,,US Dollar,...,1,BCH,Bitcoin Cash Futures,Europe/London,Bitcoin Cash Futures,EXPIRING,BCH 29 NOV 24,1198301365,2024-11,False
5,LC-29NOV24-CDE,83.2,1,0.01,0,100000000,1,100000000,,US Dollar,...,5,LTC,Litecoin Futures,Europe/London,Litecoin Futures,EXPIRING,LTC 29 NOV 24,1198301365,2024-11,False
6,ET-27DEC24-CDE,3066.5,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.1,ETH,Nano Ether Futures,Europe/London,Nano ETH,EXPIRING,ETH 27 DEC 24,3617501365,2024-12,False
7,AVA-29NOV24-CDE,32.85,1,0.01,0,100000000,1,100000000,,US Dollar,...,10,AVAX,Avalanche Futures,Europe/London,Avalanche Futures,EXPIRING,AVAX 29 NOV 24,1198301365,2024-11,False
8,NOL-19NOV24-CDE,67.33,1,0.01,0,100000000,1,100000000,,US Dollar,...,10,CDEOIL,Nano Crude Oil Futures,Europe/London,Oil Futures,EXPIRING,OIL 19 NOV 24,346901365,2024-12,False
9,SLR-27NOV24-CDE,30.425,1,0.01,0,100000000,1,100000000,,US Dollar,...,50,CDESIL,Silver Futures,Europe/London,Silver Futures,EXPIRING,SLVR 26 NOV 24,947801365,2024-12,False


In [7]:
product_frame['contract_root_unit']=product_frame['contract_root_unit'].astype(str)
futures_frame=product_frame[product_frame['contract_expiry_type']=='PERPETUAL'].copy()
# futures_frame=futures_frame[futures_frame['contract_root_unit']=='DOGE'].copy()
futures_frame['contract_expiry']=pd.to_datetime(futures_frame['contract_expiry'])
futures_frame=futures_frame.sort_values('contract_expiry',ascending=True)

In [8]:
futures_frame.reset_index(drop=True,inplace=True)
futures_frame

,product_id,price,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,base_max_size,base_name,quote_name,...,contract_size,contract_root_unit,group_description,contract_expiry_timezone,group_short_description,contract_expiry_type,contract_display_name,time_to_expiry_ms,contract_expiry_name,twenty_four_by_seven


In [9]:
future_ids=futures_frame['product_id'].unique()
future_ids

array([], dtype=object)

In [10]:
expire_ranges=futures_frame['contract_expiry'].unique()

In [11]:
futures_frame[['product_id','contract_expiry']]
futures_frame['trading_start']=futures_frame['contract_expiry'].shift(1)
futures_frame[['product_id','trading_start','contract_expiry']]
futures_frame['trading_start']=futures_frame['trading_start'].fillna(futures_frame['contract_expiry']-pd.Timedelta(days=28))
futures_frame=futures_frame[['product_id','trading_start','contract_expiry']]
futures_frame

,product_id,trading_start,contract_expiry


In [12]:
target_list=['DOGE','SHIB','ETH','BTC','SUI','LTC']
from tqdm.autonotebook import tqdm

In [13]:
start_date,end_date='2024-05-01','2024-11-12'
start_date=pd.Timestamp(start_date,tz= 'utc')
end_date=pd.Timestamp(end_date,tz= 'utc')

for idx in tqdm(future_ids,leave=True):
    symbol_base=idx.split('-')[0]
    symbol_name=symbol_base+'USD_Futures'
    if symbol_base in target_list:
        data=client.bulk_download(idx,start_date,end_date,limit=300,symbol_name=symbol_name)
        data.to_pickle(f'data_futures/{exchange}-{symbol_name}-{time_frame}.pkl')


0it [00:00, ?it/s]

In [14]:
from IPython.display import display

In [21]:
def download_futures_data(client,symbol_base='DOGE'):
    product_frame=client.get_products(product_type='FUTURE')
    product_frame['contract_root_unit']=product_frame['contract_root_unit'].astype(str)
    futures_frame=product_frame[product_frame['contract_root_unit']==symbol_base.upper()].copy()
    
    futures_frame['contract_expiry']=pd.to_datetime(futures_frame['contract_expiry'])
    futures_frame=futures_frame[futures_frame['contract_expiry_type']=='EXPIRING']
    futures_frame=futures_frame[['product_id','contract_expiry']].reset_index(drop=True)
    ascend=False
    futures_frame=futures_frame.sort_values('contract_expiry',ascending=ascend)
    futures_frame['trading_start']=futures_frame['contract_expiry'].shift(-1 if ascend==False else 1)
    futures_frame=futures_frame[['product_id','trading_start','contract_expiry']]
    futures_frame['trading_start']=futures_frame['trading_start'].fillna(futures_frame['contract_expiry']-pd.Timedelta(days=28))
    # display(futures_frame)
    print(pd.Timestamp.now(tz='utc').strftime('%Y-%m-%d %H:%M'))
    futures_frame=futures_frame[futures_frame['trading_start']<pd.Timestamp.now(tz='utc')]

    
    display(futures_frame)

    futures_data=[]
    for i,row in futures_frame.iterrows():
        symbol=row['product_id']
        start_date=row['trading_start']
        end_date=row['contract_expiry']
        
        data=client.bulk_download(symbol,start_date,end_date,limit=300)
        futures_data.append(data)
        time.sleep(1)
        print(symbol_base,start_date,end_date,len(data))

        
    futures_data=pd.concat(futures_data)
    futures_data.sort_values('date_close',inplace=True)
    futures_data=futures_data.drop_duplicates()
    return futures_data

In [22]:
start=pd.Timestamp('2024-08-30 15:00:00+00:00')
end=start+pd.Timedelta(hours=300)
start.timestamp(),end.timestamp()

(1725030000.0, 1726110000.0)

In [23]:
download_futures_data(client,'DOGE')

2024-11-15 19:08


,product_id,trading_start,contract_expiry
0,DOG-29NOV24-CDE,2024-11-01 16:00:00+00:00,2024-11-29 16:00:00+00:00


  0%|          | 0/2 [00:00<?, ?it/s]

DOGE 2024-11-01 16:00:00+00:00 2024-11-29 16:00:00+00:00 233


,date_close,low,high,open,close,volume,symbol
232,2024-11-01 16:00:00,0.16125,0.16571,0.16125,0.16413,222,DOG-29NOV24-CDE
231,2024-11-01 17:00:00,0.15695,0.16939,0.1637,0.16334,157,DOG-29NOV24-CDE
230,2024-11-01 18:00:00,0.162,0.163,0.163,0.162,109,DOG-29NOV24-CDE
229,2024-11-01 19:00:00,0.15911,0.16356,0.1618,0.16226,201,DOG-29NOV24-CDE
228,2024-11-01 20:00:00,0.16178,0.16336,0.16236,0.16228,184,DOG-29NOV24-CDE
...,...,...,...,...,...,...,...
4,2024-11-15 15:00:00,0.3625,0.37295,0.3715,0.36813,260,DOG-29NOV24-CDE
3,2024-11-15 16:00:00,0.36282,0.37466,0.36805,0.3704,255,DOG-29NOV24-CDE
2,2024-11-15 17:00:00,0.3654,0.37331,0.36928,0.3706,130,DOG-29NOV24-CDE
1,2024-11-15 18:00:00,0.36561,0.37299,0.3706,0.37022,172,DOG-29NOV24-CDE


In [24]:
account=client.account()
account



,uuid,name,default,active,type,ready,hold,retail_portfolio_id,position_value,balance
currency,,,,,,,,,,
CORECHAIN,e03f1ea5-e129-564c-889b-194077bbf294,CORECHAIN Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'CORECHAIN'}",3f98d482-321e-54af-baa5-9b5927e52212,0,5.255967
NEAR,405446ef-08b1-5205-86e4-e505ab64ae5f,NEAR Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'NEAR'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.575780
SUI,a064f48a-1267-5b19-b0ff-d754d09c6f6b,SUI Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'SUI'}",3f98d482-321e-54af-baa5-9b5927e52212,0,66.104049
DOGE,a5630355-5534-5b9d-8b26-137d6e8992dd,DOGE Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'DOGE'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.033234
ETH2,12d0e91c-efa7-52b4-87f2-2953c6ea0180,ETH2 Wallet,True,True,ACCOUNT_TYPE_CRYPTO,False,"{'value': '0', 'currency': 'ETH2'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.000298
ATOM,99b5bef3-be97-5d57-b704-68532088d286,ATOM Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'ATOM'}",3f98d482-321e-54af-baa5-9b5927e52212,0,167.549281
USDC,c041014b-790b-5f17-8469-ced1f456940d,USDC Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'USDC'}",3f98d482-321e-54af-baa5-9b5927e52212,0,470.118677
USD,cfc0be7a-b031-500b-853f-963313341334,Cash (USD),False,True,ACCOUNT_TYPE_FIAT,True,"{'value': '0', 'currency': 'USD'}",3f98d482-321e-54af-baa5-9b5927e52212,0,647.830000
ETH,9834b834-02c5-50ca-bffa-2674c9c0650e,ETH Wallet,True,True,ACCOUNT_TYPE_CRYPTO,True,"{'value': '0', 'currency': 'ETH'}",3f98d482-321e-54af-baa5-9b5927e52212,0,0.011981


In [25]:
start_time = datetime.datetime(month=11,day=1,year=2024)
end_time = datetime.datetime(month=11,day=12,year=2024)
pd.date_range(start=start_time,end=end_time,freq='1h')
def get_time_ranges(start_time,end_time,freq='1h'):
    return pd.date_range(start=start_time,end=end_time,freq=freq)


In [26]:
def download_futures(start_date='2021-01-01',end_date='2021-10-01',limit=350):
    data=client.get_historical_data(symbol=futures_target,start_date=start_date,end_date=end_date,limit=limit)
    data.to_csv('data/futures.csv',index=False)
    return data

In [27]:
product_frame=client.get_products()
product_frame

,product_id,price,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,base_max_size,base_name,quote_name,...,contract_size,contract_root_unit,group_description,contract_expiry_timezone,group_short_description,contract_expiry_type,contract_display_name,time_to_expiry_ms,contract_expiry_name,twenty_four_by_seven
0,BIT-29NOV24-CDE,90025,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.01,BTC,Nano Bitcoin Futures,Europe/London,Nano BTC,EXPIRING,BTC 29 NOV 24,1198257256,2024-11,False
1,BIT-27DEC24-CDE,90510,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.01,BTC,Nano Bitcoin Futures,Europe/London,Nano BTC,EXPIRING,BTC 27 DEC 24,3617457256,2024-12,False
2,ET-29NOV24-CDE,3050,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.1,ETH,Nano Ether Futures,Europe/London,Nano ETH,EXPIRING,ETH 29 NOV 24,1198257256,2024-11,False
3,GOL-27NOV24-CDE,2572.5,1,0.01,0,100000000,1,100000000,,US Dollar,...,1,CDEGLD,Gold Futures,Europe/London,Gold Futures,EXPIRING,GLD 26 NOV 24,948057256,2024-12,False
4,BCH-29NOV24-CDE,425.4,1,0.01,0,100000000,1,100000000,,US Dollar,...,1,BCH,Bitcoin Cash Futures,Europe/London,Bitcoin Cash Futures,EXPIRING,BCH 29 NOV 24,1198257256,2024-11,False
5,LC-29NOV24-CDE,83.2,1,0.01,0,100000000,1,100000000,,US Dollar,...,5,LTC,Litecoin Futures,Europe/London,Litecoin Futures,EXPIRING,LTC 29 NOV 24,1198257256,2024-11,False
6,ET-27DEC24-CDE,3066.5,1,0.01,0,100000000,1,100000000,,US Dollar,...,0.1,ETH,Nano Ether Futures,Europe/London,Nano ETH,EXPIRING,ETH 27 DEC 24,3617457256,2024-12,False
7,AVA-29NOV24-CDE,32.85,1,0.01,0,100000000,1,100000000,,US Dollar,...,10,AVAX,Avalanche Futures,Europe/London,Avalanche Futures,EXPIRING,AVAX 29 NOV 24,1198257256,2024-11,False
8,NOL-19NOV24-CDE,67.33,1,0.01,0,100000000,1,100000000,,US Dollar,...,10,CDEOIL,Nano Crude Oil Futures,Europe/London,Oil Futures,EXPIRING,OIL 19 NOV 24,346857256,2024-12,False
9,SLR-27NOV24-CDE,30.425,1,0.01,0,100000000,1,100000000,,US Dollar,...,50,CDESIL,Silver Futures,Europe/London,Silver Futures,EXPIRING,SLVR 26 NOV 24,947757256,2024-12,False


In [28]:
product_frame[product_frame['contract_code']=='DOG']

,product_id,price,base_increment,quote_increment,quote_min_size,quote_max_size,base_min_size,base_max_size,base_name,quote_name,...,contract_size,contract_root_unit,group_description,contract_expiry_timezone,group_short_description,contract_expiry_type,contract_display_name,time_to_expiry_ms,contract_expiry_name,twenty_four_by_seven
14,DOG-29NOV24-CDE,0.37205,1,0.01,0,100000000,1,100000000,,US Dollar,...,5000,DOGE,Dogecoin Futures,Europe/London,Dogecoin Futures,EXPIRING,DOGE 29 NOV 24,1198257256,2024-11,False
19,DOG-27DEC24-CDE,0.36668,1,0.01,0,100000000,1,100000000,,US Dollar,...,5000,DOGE,Dogecoin Futures,Europe/London,Dogecoin Futures,EXPIRING,DOGE 27 DEC 24,3617457256,2024-12,False
30,DOG-31JAN25-CDE,,1,0.01,0,100000000,1,100000000,,US Dollar,...,5000,DOGE,Dogecoin Futures,Europe/London,Dogecoin Futures,EXPIRING,DOGE 31 JAN 25,6641457256,2025-01,False


In [30]:
positions=client.trade_client.list_futures_positions()

In [ ]:
positions

In [ ]:
# products=client.trade_client.get_products()
now=datetime.datetime.now()
start_time=now-datetime.timedelta(hours=input_size*2)

int(now.timestamp())

In [ ]:
fills=client.get_orders()
fills

In [ ]:
time.sleep(10)

In [ ]:
client.sell('BTC-USDC',base_size='.000014')

In [ ]:
time.sleep(10)

In [ ]:
fills=client.get_orders()
fills